# Homework
https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/03-vector-search/homework.md

In [ ]:
# Launch ElasticSearch in Docker 
!docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [1]:
# Import libraries
from sentence_transformers import SentenceTransformer # pip install sentence_transformers==2.7.0
import requests
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get the embeddings model
https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview

In [2]:
# https://huggingface.co/sentence-transformers/multi-qa-mpnet-base-dot-v1
model = SentenceTransformer("multi-qa-distilbert-cos-v1")

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [4]:
encoded_q = model.encode(user_question)

In [11]:
# Q1. First value
encoded_q[0]

0.078222655

## Creating the embeddings

In [12]:
# Get Q&A JSON (with IDs)
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [13]:
ml_docs = [doc for doc in documents if doc['course']=='machine-learning-zoomcamp']

In [14]:
ml_docs[:3]

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp',
  'id': '0227b872'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?'

In [15]:
len(ml_docs)

375

In [16]:
#Q2. # We want to put all of them into a single matrix X:

# Create a list embeddings
# Iterate over each document
# qa_text = f'{question} {text}'
# compute the embedding for qa_text, append to embeddings
# At the end, let X = np.array(embeddings) (import numpy as np)
# What's the shape of X? (X.shape). Include the parantheses.

embeddings = []
for mldoc in tqdm(ml_docs):
    question = mldoc['question']
    text = mldoc['text']
    qa_text = f'{question} {text}'
    qa_embedding = model.encode(qa_text).tolist()
    embeddings.append(qa_embedding)
X = np.array(embeddings)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [01:04<00:00,  5.85it/s]


In [17]:
X

array([[ 0.08805905,  0.01559355,  0.07925582, ...,  0.00745118,
         0.00241914,  0.01147128],
       [ 0.07066919, -0.01930568,  0.07738373, ...,  0.04495561,
         0.01078552, -0.02316595],
       [ 0.10229158, -0.01663243,  0.03414485, ...,  0.03215314,
        -0.04317448, -0.04574372],
       ...,
       [-0.00768192,  0.0107567 ,  0.01190489, ...,  0.06066168,
        -0.03393206,  0.01605331],
       [ 0.13408281, -0.03755205,  0.01976525, ...,  0.02163627,
        -0.01921137,  0.03690192],
       [ 0.0438602 , -0.0130077 ,  0.06373122, ...,  0.03338452,
         0.0225854 , -0.06305878]])

In [18]:
X.shape

(375, 768)

## Search
cosine similarity

In [20]:
v.dot(v)

0.9999999

In [15]:
v = encoded_q
scores = X.dot(v)

In [16]:
# Q3. Highest score
round(scores.max(),4)

0.6507

In [17]:
scores.shape

(375,)

In [39]:
# Vector search
# https://github.com/DataTalksClub/llm-zoomcamp/blob/main/cohorts/2024/03-vector-search/homework.md#vector-search
# About search engine: https://github.com/alexeygrigorev/build-your-own-search-engine
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_docs, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Hit-rate for search engine

In [50]:
# loading the ground truth dataset
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [51]:
ground_truth[:2]

[{'question': 'Where can I sign up for the course?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'},
 {'question': 'Can you provide a link to sign up?',
  'course': 'machine-learning-zoomcamp',
  'document': '0227b872'}]

In [52]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [44]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = search_engine.search(model.encode(q['question']), num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:22<00:00, 22.11it/s]


In [47]:
# Q4. Hit-rate for search engine
hit_rate(relevance_total)

0.9398907103825137

## Indexing with Elasticsearch

In [ ]:
# Now let's index these documents with elasticsearch

# Create the index with the same settings as in the module (but change the dimensions)
# Index the embeddings (note: you've already computed them)
# After indexing, let's perform the search of the same query from Q1.

# What's the ID of the document with the highest score?

In [9]:
# Initiate ElasticSearch connection
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [30]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "query_text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

In [31]:
# Create indices
index_name = "course-questions"
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [33]:
# Create the dense vector using the already calculated vectors from pre-trained model
operations = []
for doc, vec in zip(ml_docs, embeddings):
    try:
        doc['query_text_vector'] = vec
        operations.append(doc)
    except Exception as e:
        print(e)

In [34]:
# Index dataset
for doc in tqdm(operations):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:12<00:00, 31.07it/s]


In [66]:
def elastic_search(query, index='course-questions'):
    index_name = index
    vector_search_term = model.encode(query)
    
    query = {
    "field": "query_text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000,
        }

    response = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course", "id"])
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [67]:
user_question

'I just discovered the course. Can I still join it?'

In [68]:
elastic_search(user_question)[0]

{'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'section': 'General course-related questions',
 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'id': 'ee58a693'}

In [69]:
# Q5. ID with the highest score
elastic_search(user_question)[0]['id']

'ee58a693'

## Hit-rate for Elasticsearch

In [71]:
es_relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], index=index_name)
    relevance = [d['id'] == doc_id for d in results]
    es_relevance_total.append(relevance)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:24<00:00, 21.54it/s]


In [73]:
# Q6. Hit-rate for ElasticSearch
hit_rate(es_relevance_total)

0.9398907103825137